# 损失函数类

损失函数是前向传播的终点，也是**计算图**（Computational Graph）的终点，还也是梯度计算的起点。

## 计算图

我们已经知道，人工神经元网络模型并不是一次性地从输入数据计算出输出数据。而是输入数据经过一层一层的神经元，通过多次（简单）计算最终得出输出数据。这个数据流通的线路，被称为前向传播。

在前向传播的过程中，我们需要记录下来输入数据流通的整个拓扑结构。因为我们需要通过这个拓扑结构反向地逐级计算梯度。这个数据前向传播的拓扑结构，被称为计算图。因为它是在前向传播的过程中动态构建起来的，所以称为**动态计算图**（Dynamic Computational Graph）。

相对应的，在前向传播进行前，就先构建好的计算图，称为**静态计算图**（Static Computational Graph）。

PyTorch采用动态计算图技术，而TensorFlow则采用静态计算图技术。

我们也将通过张量来实现动态计算图。

In [1]:
from abc import abstractmethod, ABC
import numpy as np

## 基础架构

### 张量

In [2]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)

    @property
    def size(self):
        return len(self.data)

    def __repr__(self):
        return f'Tensor({self.data})'

### 基础层

In [3]:
class Layer(ABC):

    def __call__(self, x: Tensor):
        return self.forward(x)

    @abstractmethod
    def forward(self, x: Tensor):
        pass

    def __repr__(self):
        return ''

### 基础损失函数

和基础层一样，基础损失函数也是一个抽象类，定义了一个计算梯度的接口。这个接口需要返回一个损失值的张量，作为梯度计算的起点。

In [4]:
class Loss(ABC):

    def __call__(self, p: Tensor, y: Tensor):
        return self.loss(p, y)

    @abstractmethod
    def loss(self, p: Tensor, y: Tensor):
        pass

## 数据

### 特征、标签

In [5]:
feature = Tensor([28.1, 58.0])
label = Tensor([165])

## 模型

### 线性层

In [6]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        self.weight = Tensor(np.ones((out_size, in_size)) / in_size)
        self.bias = Tensor(np.zeros(out_size))

    def forward(self, x: Tensor):
        return Tensor(x.data @ self.weight.data.T + self.bias.data)

    def __repr__(self):
        return f'Linear[weight{self.weight.data.shape}; bias{self.bias.data.shape}]'

### 损失函数（平均平方差）

损失函数（平均平方差）继承了基础损失函数的虚拟接口，封装了计算平均平方差的功能。这是回归模型最常用的损失函数。

In [7]:
class MSELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        return Tensor(np.mean(np.square(y.data - p.data)))

## 验证

### 推理

In [8]:
layer = Linear(feature.size(), 1)
prediction = layer(feature)
print(f'prediction: {prediction}')

prediction: Tensor([43.05])


### 评估

In [9]:
loss_fn = MSELoss()
loss = loss_fn(prediction, label)
print(f'loss: {loss}')

loss: Tensor(14871.802500000002)
